In [4]:
import numpy as np
import pyskip

In [35]:
def transpose(mat):
    if len(mat) <= 1024 * 1024:
        return pyskip.Tensor.from_numpy(mat.to_numpy().transpose())
    cols, rows = mat.shape
    assert cols & (cols - 1) == 0
    assert rows & (rows - 1) == 0
    sub_11 = transpose(mat[:cols // 2, :rows // 2])
    sub_12 = transpose(mat[:cols // 2, rows // 2:])
    sub_21 = transpose(mat[cols // 2:, :rows // 2])
    sub_22 = transpose(mat[cols // 2:, rows // 2:])

    ret = pyskip.Tensor(shape=(cols, rows), dtype=mat.dtype)
    ret[:cols // 2, :rows // 2] = sub_11
    ret[:cols // 2, rows // 2:] = sub_21
    ret[cols // 2:, :rows // 2] = sub_12
    ret[cols // 2:, rows // 2:] = sub_22
    return ret

In [36]:
x = pyskip.Tensor.from_numpy(
    np.arange(4096 * 4096).reshape(4096, 4096)
)

In [37]:
%%time

y = transpose(x)

Wall time: 875 ms


In [42]:
%%time

mat = x.clone()
cols, rows = mat.shape

sub_11 = (2 * mat[:cols // 2, :rows // 2]).eval()
sub_12 = (2 * mat[:cols // 2, rows // 2:]).eval()
sub_21 = (2 * mat[cols // 2:, :rows // 2]).eval()
sub_22 = (2 * mat[cols // 2:, rows // 2:]).eval()

ret = pyskip.Tensor(shape=(cols, rows), dtype=mat.dtype)
ret[:cols // 2, :rows // 2] = sub_11
ret[:cols // 2, rows // 2:] = sub_21
ret[cols // 2:, :rows // 2] = sub_12
ret[cols // 2:, rows // 2:] = sub_22

Wall time: 107 ms


In [44]:
%%time
mat_2 = mat.clone().eval()

Wall time: 82 ms
